In [1]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import torch.nn.functional as F
from skopt import BayesSearchCV
from skopt.space import Real
from skopt import gp_minimize
import torch.nn as nn
import seaborn as sns
from scipy.optimize import minimize
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib as mpl
from skopt.utils import use_named_args
from tqdm import tqdm
import multiprocessing
import threading

In [2]:
# 设置GPU设备，如果没有可用的GPU，则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(888888)

print(device)

cpu


## 基础参数

In [3]:
# 基础参数
epochs = 400000    # 训练代数
h = 100    # 画图网格密度
N = 100    # 内点配置点数
N1 = 0.01    # 边界点配置点数
n = 1000    # PDE数据点
top_k = 100  # 前n个残差最大的点
num_new_points = 10  # 以圆心生成的n个点
bias = 0.001   # 圆半径

## PINN框架

In [4]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

## 损失函数

In [5]:
# Loss
loss = torch.nn.MSELoss()
# loss = torch.nn.L1Loss()  #MAE
# loss = torch.nn.SmoothL1Loss(beta=1.0)

# 递归求导
def gradients(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, )[0]
    else:
        return gradients(gradients(u, x), x, order=order - 1)

In [6]:
X = -torch.rand(n, 1, device=device, requires_grad=True)
Y = -torch.rand(n, 1, device=device, requires_grad=True)

def point(n=100, device=device):
    # 随机生成 n 个点
    x = X
    y = Y
    return x, y

def l_ugeq(u, n=100):
    # u >= 0
    x, y = point(n, device)
    cond = torch.zeros_like(y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    llos = torch.relu(-uxy)
    ugeq_loss = 100 * torch.sum(llos)
    
    # 返回每个点的损失值
    return ugeq_loss, llos, x, y

def l_pde(u, n=100, device=device):
    # 等式项损失
    x, y = point(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
    
    # 计算损失
    uxy = u(torch.cat([x, y], dim=1))
    
    # 确保梯度计算返回每个点的值
    grad_x2 = gradients(uxy, x, 2)  # 对 x 计算二阶梯度
    grad_y2 = gradients(uxy, y, 2)  # 对 y 计算二阶梯度

    # 计算每个点的损失
    loss_value = -grad_x2 - grad_y2  # 假设这里是 PDE 中的计算
    loss_value = loss_value - cond   # 计算 PDE 的残差
    pde_loss = loss(-gradients(uxy, x, 2) - gradients(uxy, y, 2)- cond, torch.zeros_like(cond))

    # 返回每个点的损失值
    return pde_loss, loss_value, x, y

# def l_JU(u, n=100):
#     # 不等式损失
#     x, y = point(n, device)
#     cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)
    
#     # 计算损失
#     uxy = u(torch.cat([x, y], dim=1))
#     llos = 0.5 * (gradients(uxy, x, 1)**2 + gradients(uxy, y, 1)**2) - cond * uxy
    
#     JU_loss = loss(0.5 * (gradients(uxy, x, 1)**2 + gradients(uxy, y, 1)**2), cond * uxy)
    
#     # 返回每个点的损失值
#     return JU_loss, llos, x, y


In [7]:
def l_JU(u, n=100, device=device):
    """
    计算每个点的 JU 损失值和整体的 JU 损失（通过蒙特卡罗积分近似）
    
    参数：
    - u: 神经网络
    - n: 采样点的数量
    - device: 设备（"cuda" 或 "cpu"）
    
    返回：
    - JU_loss: 总的目标函数值（通过积分近似）
    - llos_JU: 每个点的损失值（对应每个采样点）
    - x, y: 对应每个点的坐标
    """
    # 随机采样点
    x, y = point(n, device)
    cond = 2 * torch.pi**2 * torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

    # 计算 u 和其梯度
    uxy = u(torch.cat([x, y], dim=1))
    grad_x = gradients(uxy, x, 1)
    grad_y = gradients(uxy, y, 1)
    
    # 每个点的损失值
    llos_JU = 0.5 * (grad_x**2 + grad_y**2) - cond * uxy  # 每个点对应的损失值
    p1 = 0.5 * (grad_x**2 + grad_y**2)
    p2 = cond * uxy

    # 使用蒙特卡罗积分近似整体损失
    JU_loss = llos_JU.mean()  # 取所有点的平均值作为积分的近似

    return JU_loss, llos_JU, x, y, p1, p2


In [8]:
# 边界函数构建
def l_boundary1(u):
    # 取点
    x = -torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = -torch.ones_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary2(u):
    # 取点
    y = -torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = -torch.ones_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary3(u):
    # 取点
    y = -torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    x = torch.zeros_like(y, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

def l_boundary4(u):
    # 取点
    x = -torch.arange(0, 1.001, N1, device=device, requires_grad=True).view(-1, 1)
    y = torch.zeros_like(x, device=device, requires_grad=True)
    cond = torch.zeros_like(x, device=device)
    # 计算
    uxy = u(torch.cat([x, y], dim=1))
    return loss(uxy, cond)

## 数据集更新

In [9]:
def update_dataset(X, Y, inside_loss, x_pde, y_pde, top_k=10, num_new_points=10, bias=0.001, device=device):
    """
    更新数据集，包括：
    - 找到前 top_k 个高损失点
    - 以高损失点为圆心生成随机新点
    - 删除高损失点
    - 去重数据集
    
    参数:
    X (Tensor): 当前数据集的 X 坐标
    Y (Tensor): 当前数据集的 Y 坐标
    inside_loss (Tensor): 每个点的损失值
    x_pde (Tensor): PDE 数据的 X 坐标
    y_pde (Tensor): PDE 数据的 Y 坐标
    top_k (int): 选择前 top_k 个高损失点
    num_new_points (int): 每个高损失点生成的新点数
    bias (float): 新点生成的半径
    device (str): 设备类型 ("cpu" 或 "cuda")

    返回:
    X (Tensor): 更新后的 X 坐标
    Y (Tensor): 更新后的 Y 坐标
    """
    with torch.no_grad():  # 禁用梯度计算
        # 获取绝对损失值
        abs_loss = torch.abs(inside_loss)  # 对每个点的损失取绝对值

        # 将损失值和对应的 x, y 坐标组合为一个元组 (loss, x, y)
        loss_coords = list(zip(abs_loss.cpu().numpy(), x_pde.cpu().numpy(), y_pde.cpu().numpy()))

        # 按照损失值进行排序（按损失的绝对值降序）
        sorted_loss_coords = sorted(loss_coords, key=lambda x: x[0], reverse=True)

        # 获取前 top_k 个高损失点
        top_k_x = [sorted_loss_coords[i][1] for i in range(top_k)]
        top_k_y = [sorted_loss_coords[i][2] for i in range(top_k)]
        
        top_k_x_tensor = torch.tensor(top_k_x, device=device).view(-1, 1)
        top_k_y_tensor = torch.tensor(top_k_y, device=device).view(-1, 1)

        # 从 X 和 Y 中删除这些点
        mask = torch.ones(X.shape[0], dtype=torch.bool, device=device)
        for i in range(top_k):
            mask &= ~((X == top_k_x_tensor[i]).view(-1) & (Y == top_k_y_tensor[i]).view(-1))
        X = X[mask]
        Y = Y[mask]

        # 为每个点生成附近的随机点
        all_new_x = []
        all_new_y = []
        for i in range(top_k):
            center_x = top_k_x_tensor[i]
            center_y = top_k_y_tensor[i]

            # 随机生成 n 个附近点
            angles = torch.rand(num_new_points, device=device) * 2 * torch.pi
            radii = torch.sqrt(torch.rand(num_new_points, device=device)) * bias
            offset_x = radii * torch.cos(angles)
            offset_y = radii * torch.sin(angles)

            new_points_x = center_x + offset_x
            new_points_y = center_y + offset_y

            all_new_x.append(new_points_x)
            all_new_y.append(new_points_y)

        # 将所有新点的 x 和 y 坐标合并
        all_new_x = torch.cat(all_new_x, dim=0).view(-1, 1)
        all_new_y = torch.cat(all_new_y, dim=0).view(-1, 1)

        # 将新点添加回数据集
        X = torch.cat([X, all_new_x], dim=0)
        Y = torch.cat([Y, all_new_y], dim=0)

        # 去除重复点
        points = torch.cat([X, Y], dim=1)  # 合并为 (x, y) 点对
        points = torch.unique(points, dim=0)  # 按行去重
        X, Y = points[:, 0].view(-1, 1), points[:, 1].view(-1, 1)  # 拆分回 X 和 Y

    # 返回时重新启用 requires_grad
    return X.requires_grad_(True), Y.requires_grad_(True)


## 查看情况

In [10]:
def record_loss_and_save_model(u, epoch, maxerror, current_time, h, device):
    """
    每100次迭代记录一次损失并保存模型。

    参数:
    - u: 模型
    - epoch: 当前训练的 epoch 数
    - maxerror: 当前最小的误差
    - current_time: 上一次记录的时间
    - h: 网格划分大小
    - device: 设备
    返回:
    - maxerror: 更新后的最小误差
    - current_time: 更新后的时间
    """
    # 生成网格点
    xc_x = -torch.linspace(0, 1, h, device=device)
    xc_y = -torch.linspace(0, 1, h, device=device)
    xm, ym = torch.meshgrid(xc_x, xc_y)
    xx = xm.reshape(-1, 1)
    yy = ym.reshape(-1, 1)
    xy = torch.cat([xx, yy], dim=1).to(device)

    # 计算预测值和真实值
    u_pred = u(xy)
    u_real = torch.relu(torch.sin(torch.pi * xx) * torch.sin(torch.pi * yy))
    u_error = torch.abs(u_pred - u_real)

    # 计算误差网格
    u_pred_fig = u_pred.reshape(h, h)
    u_real_fig = u_real.reshape(h, h)
    u_error_fig = u_error.reshape(h, h)

    # 计算当前最大绝对误差
    max_abs_error = float(torch.max(u_error))
    print(f"At epoch {epoch}, time_speed: {abs(current_time - time.time()):.2f}s, Max abs error is: {max_abs_error}, best: {maxerror}")
    print('-----------------------------------------------------------------------------------------------------------------------')

    # 如果误差更小，则保存模型
    if max_abs_error < maxerror:
        maxerror = max_abs_error
        torch.save(u.state_dict(), 'weights11.pth')

    # 更新当前时间
    current_time = time.time()

    return maxerror, current_time


## 训练

In [ ]:
# Training
u = MLP().to(device)
opt = torch.optim.Adam(params=u.parameters(), lr=0.0001)

# 设置初始权重
weight_pde = 60000
weight_JU = 2000
weight_boundary = 10000000
current_time = time.time()
maxerror = 99999999999

for epoch in range(epochs):
    opt.zero_grad()

    # # 手动调整学习率
    # if epoch == 100000:
    #     for param_group in opt.param_groups:
    #         param_group['lr'] *= 5  # 学习率放大 1/10 倍
    
    # 计算各个损失函数并获取点
    pde_loss, llos_pde, x_pde, y_pde = l_pde(u)
    JU_loss, llos_JU, x_JU, y_JU, p1, p2 = l_JU(u)
    ugeq_loss, llos_ugeq, x_ugeq, y_ugeq = l_ugeq(u)
    
    inside_loss = weight_pde * llos_pde + weight_JU * llos_JU + 6000 * llos_ugeq
    loss_boundary = l_boundary1(u) +  l_boundary2(u) + l_boundary3(u) + l_boundary4(u)
    
    # 计算加权总损失
    total_loss = weight_pde * pde_loss + weight_JU * JU_loss + 600 * ugeq_loss + weight_boundary * loss_boundary
    # total_loss = JU_loss
    # 每100次迭代记录一次损失
    if epoch % 100 == 0 and epoch != 0:
        print(f"pde:{pde_loss.item()},JU:{JU_loss.item()},boundary:{loss_boundary.item()}")
        print(p1.mean().item(), p2.mean().item())
        maxerror, current_time = record_loss_and_save_model(u, epoch, maxerror, current_time, h, device)
        
    # if epoch % 100000 == 0 and epoch != 0:
    #     X, Y = update_dataset(X, Y, inside_loss, x_pde, y_pde, top_k, num_new_points, bias, device)
    
    # 反向传播和优化
    total_loss.backward()
    opt.step()



pde:106.59680938720703,JU:-0.02830650843679905,boundary:0.00020675777341239154
8.016268111532554e-05 0.028386669233441353
At epoch 100, time_speed: 4.97s, Max abs error is: 0.9973300695419312, best: 99999999999
-----------------------------------------------------------------------------------------------------------------------


/root/miniconda3/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


pde:105.65245056152344,JU:-0.05078580603003502,boundary:0.00012699447688646615
1.999922278628219e-05 0.05080581083893776
At epoch 200, time_speed: 4.12s, Max abs error is: 0.9933456778526306, best: 0.9973300695419312
-----------------------------------------------------------------------------------------------------------------------
pde:103.86475372314453,JU:-0.11249706894159317,boundary:0.00020126128219999373
0.00031285558361560106 0.1128099262714386
At epoch 300, time_speed: 2.29s, Max abs error is: 0.9834954738616943, best: 0.9933456778526306
-----------------------------------------------------------------------------------------------------------------------
pde:98.85189819335938,JU:-0.3184211850166321,boundary:0.0009463807800784707
0.0035981929395347834 0.3220193386077881
At epoch 400, time_speed: 1.98s, Max abs error is: 0.9514203071594238, best: 0.9834954738616943
-----------------------------------------------------------------------------------------------------------------